In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
search = "music" #hashtag, creator

URL = f"https://ads.tiktok.com/business/creativecenter/inspiration/popular/{search}/pc/en"
page = requests.get(URL)

soup = BeautifulSoup(page.text)
scripts = soup.find("script", id='__NEXT_DATA__')

In [4]:
search = "hashtag" #music
hashtag = "jhope"
period = "120"
country = "FR"

URL = f"https://ads.tiktok.com/business/creativecenter/{search}/{hashtag}/pc/en?countryCode={country}&period={period}"
page = requests.get(URL)

soup = BeautifulSoup(page.text)
script = soup.find('meta', content="width=device-width")

In [7]:
URL = f"https://ads.tiktok.com/business/creativecenter/trends/home/pc/en"
page = requests.get(URL)

soup = BeautifulSoup(page.text)

In [3]:
trending_hashtag = []
trending_music = []
trending_creator = []

for s in soup.find_all('div', id='hashtagItemContainer'):
    tag = s.find('span', {"class": "titleText--qKHbP"}).text
    stats = s.find_all('span', {"class": "item-value--VAdnq"})
    posts = stats[0].text
    views = stats[1].text
    trending_hashtag.append([tag, posts, views])

for s in soup.find_all('div', {"class": "music-name-wrap--FPaLL music-name-wrap--X6m9u"}):
    trending_music.append(s.text)
    
for s in soup.find_all('div', id='creatorItemContainer'):
    tag = s.find('span', {"class": "music-name--LR+1s music-name--bEzZ1"}).text
    stats = s.find_all('span', {"class": "creator-data-value--CwFQt creator-data-value--S-K2V"})
    posts = stats[0].text
    views = stats[1].text
    trending_creator.append([tag, posts, views])

hashtag = pd.DataFrame(trending_hashtag)
music = pd.DataFrame(trending_music)
creator = pd.DataFrame(trending_creator)  

In [6]:
creator

,0,1,2
0,MrBeast,77M,724M
1,avani,43M,3B
2,XO Team,40M,1B
3,James Charles,38M,1B
4,noah beck,34M,2B


In [8]:
find_content = soup.find('script', id="__NEXT_DATA__" )
str_content = str(find_content).split('<')[1].split('>')[1]
json_content = json.loads(str_content)['props']['pageProps']['data']

In [9]:
hashtags_content = json_content['hashtags']
tiktoks_content = json_content['tiktoks']
trendMusic_content = json_content['trendMusic']
creatorList_content = json_content['creatorList']

In [4]:
import subprocess
import json
import pandas as pd
import requests, ast, re
from bs4 import BeautifulSoup


def fetch_top_trends():
    
    URL = f"https://ads.tiktok.com/business/creativecenter/trends/home/pc/en"
    page = requests.get(URL)
    soup = BeautifulSoup(page.text)

    trending_hashtag = []
    trending_music = []
    trending_creator = []

    for s in soup.find_all('div', id='hashtagItemContainer'):
        tag = s.find('span', {"class": "titleText--qKHbP"}).text
        stats = s.find_all('span', {"class": "item-value--VAdnq"})
        posts = stats[0].text
        views = stats[1].text
        trending_hashtag.append([tag, posts, views])

    for s in soup.find_all('div', {"class": "music-name-wrap--FPaLL music-name-wrap--X6m9u"}):
        trending_music.append(s.text)

    for s in soup.find_all('div', id='creatorItemContainer'):
        tag = s.find('span', {"class": "music-name--LR+1s music-name--bEzZ1"}).text
        stats = s.find_all('span', {"class": "creator-data-value--CwFQt creator-data-value--S-K2V"})
        posts = stats[0].text
        views = stats[1].text
        trending_creator.append([tag, posts, views])

    hashtag = pd.DataFrame(trending_hashtag, columns = ['tag', 'posts', 'views'])
    music = pd.DataFrame(trending_music, columns = ['music'])
    creator = pd.DataFrame(trending_creator, columns = ['creator', 'followers', 'likes'])
    
    find_content = soup.find('script', id="__NEXT_DATA__" )
    str_content = str(find_content).split('<')[1].split('>')[1]
    json_content = json.loads(str_content)['props']['pageProps']['data']
    
    hashtags_content = json_content['hashtags']
    tiktoks_content = json_content['tiktoks']
    trendMusic_content = json_content['trendMusic']
    creatorList_content = json_content['creatorList']

    
    hashtag = hashtag.merge(pd.DataFrame.from_records(hashtags_content), left_index=True, right_index=True)
    music = music.merge(pd.DataFrame.from_records(trendMusic_content), left_index=True, right_index=True)
    creator = creator.merge(pd.DataFrame.from_records(creatorList_content), left_index=True, right_index=True)
    tiktok = pd.DataFrame.from_records(tiktoks_content)

    return hashtag, music, creator, tiktok


In [5]:
hashtag, music, creator, tiktok = fetch_top_trends()

In [6]:
music

,music,author,cover,link,promoted,rank,title,trend,clipId,countryCode,ifCml,isSearch,onListTimes,rankDiff,rankDiffType,songId,urlTitle
0,Big Boy Speed Up,Dj Kampung,https://p16-sg.tiktokcdn.com/aweme/100x100/tos...,https://www.tiktok.com/music/x-719395921411144...,False,1,Big Boy Speed Up,"[{'time': 1676419200, 'value': 0.35}, {'time':...",7193959214111442946,US,False,False,None,1.0,1,7193959213800884225,Big-Boy-Speed-Up
1,Boy's a liar Pt. 2,PinkPantheress & Ice Spice,https://p16-sg.tiktokcdn.com/aweme/100x100/tos...,https://www.tiktok.com/music/x-719702572368845...,False,2,Boy's a liar Pt. 2,"[{'time': 1676419200, 'value': 0.25}, {'time':...",7197025723688454146,US,False,False,None,1.0,1,7189713675082303490,Boy-s-a-liar-Pt-2
2,The Freshmen,The Verve Pipe,https://p16-sg.tiktokcdn.com/aweme/100x100/tos...,https://www.tiktok.com/music/x-223207947944730624,False,3,The Freshmen,"[{'time': 1676419200, 'value': 0}, {'time': 16...",223207947944730624,US,False,False,None,24.0,1,6696822281877850114,The-Freshmen
3,The Perfect Girl,Mareux,https://p16-sg.tiktokcdn.com/aweme/100x100/tos...,https://www.tiktok.com/music/x-684136678467148...,False,4,The Perfect Girl,"[{'time': 1676419200, 'value': 0}, {'time': 16...",6841366784671483906,US,False,False,None,NaN,4,7041352781252265985,The-Perfect-Girl
4,Hey Hey - Radio Edit,Polo Frost,https://p16-sg.tiktokcdn.com/aweme/100x100/tos...,https://www.tiktok.com/music/x-719101761399501...,False,5,Hey Hey - Radio Edit,"[{'time': 1676419200, 'value': 0.01}, {'time':...",7191017613995018241,US,False,False,None,3.0,1,7191016637745596417,Hey-Hey-Radio-Edit
